In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils import data

import argparse


from time import time, sleep
import datetime
import os


In [36]:
class DataLoader(data.Dataset):
    def __init__(self, fpath, max_len, train_mode=False):
        entries = open(fpath, 'r').readlines()
        self.x_list, self.y_list = [], []
        self.max_len = max_len
       
        for entry in entries:
            entry = entry.strip()
            x = [int(x) for x in entry.split()[:-1]]
            if train_mode==True:
                y = [int(y) for y in entry.split()[-1]]
            else:
                y = [0]
            self.x_list.append(x)
            self.y_list.append(y)
        self.x_list = torch.FloatTensor(self.x_list)
        self.y_list = torch.Tensor(self.y_list)

    def __len__(self):
        return len(self.x_list)

    def __getitem__(self, idx):
        x, y = self.x_list[idx], self.y_list[idx]
        if len(x) > self.max_len:
            x = x[:self.max_len]
        if len(y) > self.max_len:
            y = y[:self.max_len]

        x_seqlen, y_seqlen = len(x), len(y)

        return x, x_seqlen, y, y_seqlen


In [3]:
import torch
import torch.nn as nn

class MLP_model(nn.Module):
    def __init__(self, device='cpu', input_size=None, output_size=None, dr=0.0):
        super().__init__()
        self.fc1 = nn.Linear(input_size, output_size)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p=dr)
        self.device = device

    def forward(self, x, y, ):
        x = x.to(self.device)
        y = y.to(self.device)

        logits = self.fc1(x)
        logits = self.sigmoid(logits)

        y_hat = logits.argmax(-1)

        return logits, y, y_hat


In [23]:
def train(model, iterator, optimizer, criterion):
    model.train()
    for i, batch in enumerate(iterator):
        x, x_seqlens, y, y_seqlens = batch

        optimizer.zero_grad()

        logits, y, y_hat = model(x, y)
        loss = criterion(logits, y)
        loss.backward()

        optimizer.step()

    return y, logits, loss.item()

In [41]:
def pred(model, iterator):
    model.eval()
    
    for i, batch in enumerate(test_iter):
        x, x_seqlens, y, y_seqlens = batch
        logits, _, _ = model(x,y)
        print(f"input: {x.cpu().tolist()}")
        print(f"result: {logits.cpu().tolist()}")
        

In [43]:

parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument("--lr", type=float, default=0.1)
parser.add_argument("--n_epochs", type=int, default=1000)
parser.add_argument("--input_size", type=int, default=2)
parser.add_argument("--output_size", type=int, default=1)
parser.add_argument("--max_len", type=int, default=2)
parser.add_argument("--dropout", type=float, default=0.1)
parser.add_argument("--logdir", type=str, default="checkpoints")
parser.add_argument("--trainset", type=str, default="data/and.txt")
parser.add_argument("--model_path", type=str, default="checkpoints/best.pt")
args = parser.parse_args(args=[])

train_dataset = DataLoader(args.trainset, args.max_len, train_mode=True)
train_iter = data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP_model(device=device, input_size=args.input_size, output_size=args.output_size, dr=args.dropout)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=args.lr)
criterion = nn.BCELoss()

for epoch in range(1, args.n_epochs+1):
    y, logits, loss = train(model, train_iter, optimizer, criterion)

    if epoch%100==0:
        print(f'train in {epoch}...')
        print(f'y: {y}')
        print(f"y': {logits}")
        print(f"loss: {loss}")
        sleep(3)
torch.save(model, args.model_path)
print(f'training result')
print(f'y: {y}')
print(f"y': {logits}")


save = open('data/test.txt','w')
save.write('0 1 -1')
save.close()

test_dataset = DataLoader('data/test.txt', args.max_len, train_mode=False)
test_iter = data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP_model(device=device, input_size=args.input_size, output_size=args.output_size, dr=args.dropout)
model = torch.load(args.model_path)
model.to(device)

pred(model, test_iter)

train in 100...
y: tensor([[0.],
        [0.],
        [0.],
        [1.]])
y': tensor([[0.1231],
        [0.1564],
        [0.3443],
        [0.5950]], grad_fn=<SigmoidBackward0>)
loss: 0.31065186858177185
train in 200...
y: tensor([[0.],
        [0.],
        [0.],
        [1.]])
y': tensor([[0.0252],
        [0.0894],
        [0.1849],
        [0.8093]], grad_fn=<SigmoidBackward0>)
loss: 0.13382743299007416


KeyboardInterrupt: 